In [1]:
import numpy as np
import pandas as pd
import PIL
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,LocallyConnected2D,BatchNormalization
from tensorflow.keras.preprocessing import image

In [2]:
nRowsRead = 10000 # specify 'None' if want to read whole file
# list_bbox_celeba.csv has 202599 rows in reality, but we are only loading/previewing the first 1000 rows
df = pd.read_csv('list_bbox_celeba.csv', delimiter=',', nrows = nRowsRead)
df.dataframeName = 'list_bbox_celeba.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 10000 rows and 5 columns


In [3]:
df.columns

Index(['image_id', 'x_1', 'y_1', 'width', 'height'], dtype='object')

In [4]:
images_data_path = "C:/Users/foreh/Documents/celebaRepo/FacialAttrCNN/celeba/greyscale"

In [5]:
width = 178
height = 218
image_size = (width,height)
images_data = list()
for idx in range(df.shape[0]):
    path = "{}/{}".format(str(images_data_path),str(df.iloc[idx].image_id))
    image = PIL.Image.open(path)
    image_array = np.asarray(image)
    images_data.append(image_array)
images_data = np.array(images_data)

In [6]:
images = images_data # list of array of images
labels = df[["image_id", "x_1", "y_1", "width", "height"]]  # dataframe of image features
train_images,test_images,train_labels,test_labels = train_test_split(images,labels,test_size=0.3,random_state=(6))
y_test=test_labels.drop(["image_id"],axis=1)
y_train=train_labels.drop(["image_id"],axis=1)
X_test=test_images
X_train=train_images
#X_train = tf.expand_dims(X_train, axis=-1)
print("Samples Training Set:",len(X_train))
X_test, X_val, y_test, y_val=train_test_split(X_test, y_test,test_size=0.5,random_state=(5))
print("Samples in Validation Data:",len(X_val))
print("Samples in Test Data:",len(X_test))

Samples Training Set: 7000
Samples in Validation Data: 1500
Samples in Test Data: 1500


In [26]:
model = Sequential()
model.add(tf.keras.Input(shape=(image_size[1],image_size[0], 1)))
model.add(Conv2D(24, (3,3), strides=(3,3),padding="valid"))
model.add(Conv2D(8, (3,3), strides=(2,1)))
model.add(MaxPooling2D(2))
model.add(Dropout(.2))
model.add(BatchNormalization())
model.add(Conv2D(16, (3,3)))
model.add(MaxPooling2D(2))
model.add(BatchNormalization())
model.add(Conv2D(8, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(44))
model.add(Dense(32))
model.add(Dense(4, activation="linear"))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 72, 59, 24)        240       
                                                                 
 conv2d_19 (Conv2D)          (None, 35, 57, 8)         1736      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 17, 28, 8)        0         
 g2D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 17, 28, 8)         0         
                                                                 
 batch_normalization_5 (Batc  (None, 17, 28, 8)        32        
 hNormalization)                                                 
                                                                 
 conv2d_20 (Conv2D)          (None, 15, 26, 16)       

In [27]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(optimizer = opt,loss="huber_loss",metrics = ['acc', 'mse'])

In [28]:
training_process = model.fit(X_train,y_train,epochs = 10,validation_data = (X_val,y_val),batch_size=8,shuffle = True)

Epoch 1/10
875/875 [==============================] - 14s 15ms/step - loss: 91.1334 - acc: 0.7514 - mse: 29114.8281 - val_loss: 82.0467 - val_acc: 0.8107 - val_mse: 23523.3301
Epoch 2/10
875/875 [==============================] - 13s 15ms/step - loss: 77.9082 - acc: 0.8149 - mse: 23969.8613 - val_loss: 75.8487 - val_acc: 0.8107 - val_mse: 21200.5312
Epoch 3/10
875/875 [==============================] - 13s 15ms/step - loss: 73.4943 - acc: 0.8149 - mse: 22026.9160 - val_loss: 67.8702 - val_acc: 0.8107 - val_mse: 16039.3281
Epoch 4/10
875/875 [==============================] - 12s 14ms/step - loss: 71.3093 - acc: 0.8149 - mse: 21370.4512 - val_loss: 69.7414 - val_acc: 0.8107 - val_mse: 18850.3848
Epoch 5/10
875/875 [==============================] - 13s 15ms/step - loss: 70.3898 - acc: 0.8149 - mse: 20922.8613 - val_loss: 67.6960 - val_acc: 0.8107 - val_mse: 14927.2744
Epoch 6/10
875/875 [==============================] - 13s 15ms/step - loss: 69.4683 - acc: 0.8149 - mse: 20601.5938 - va